In [ ]:
#to open cred files
import os
#cleaning and operations
import pandas as pd
import numpy as np
#to read translations
import json
import unidecode

In [ ]:
#bigquery
import pandas_gbq
from google.cloud import bigquery

In [ ]:
#FIREBASE
from firebase_admin import credentials,storage,firestore,initialize_app

In [ ]:
from dotenv import load_dotenv
load_dotenv('/Users/daniel/OAN/credentials/contoan/.env')

In [ ]:
## OPEN CONECTION TO BIGQUERRY
client = bigquery.Client()
dataset_id = "{}.contoan".format(client.project)
dataset = bigquery.Dataset(dataset_id)

In [ ]:
filename=os.environ['FIREBASE_FILENAME']
#OPEN CONECTION TO FIREBASE
cred = credentials.Certificate(filename)
firebase = initialize_app(cred)

db = firestore.client()

## CONNCET TO SUMUP

In [ ]:
import requests

In [ ]:
creds_sumup = json.load(open(sumup_file,'r'))

In [ ]:
authorize_url = creds_sumup["authorize_url"]
token_url = creds_sumup["token_url"]

In [ ]:
client_id = creds_sumup["client_id"]
client_secret = creds_sumup["client_secret"]

In [ ]:
callback_uri = creds_sumup["redirect_uris"]

In [ ]:
scopes = ['transactions.history','user.profile','user.profile_readonly']

In [ ]:
trans_hist = "https://api.sumup.com/v0.1/me/transactions/history"

In [ ]:
authorization_redirect_url = (authorize_url +
                              '?response_type=code&client_id=' +
                              client_id +
                              '&redirect_uri=' + 
                              callback_uri[0] +
                              '&scope='+scopes[0])

In [ ]:
print(authorization_redirect_url)

In [ ]:
data = {'grant_type': 'authorization_code',
        'code': "#COPIAR DE LA URL ANTERIOR#",
        'redirect_uri': callback_uri}
access_token_response = requests.post(token_url,
                                      data=data,
                                      verify=False,
                                      allow_redirects=False,
                                      auth=(client_id, client_secret))


In [ ]:
tokens = json.loads(access_token_response.text)
access_token = tokens['access_token']

In [ ]:
api_call_headers = {'Authorization': 'Bearer ' + access_token}
params = {
    "limit":1000
}
api_call_response = requests.get(trans_hist, 
                                 headers=api_call_headers, 
                                 params=params,
                                 verify=False)

In [ ]:
api_call_response.status_code

In [ ]:
ventas_sumup = api_call_response.json()

In [ ]:
ventas_sumup = ventas_sumup['items']

In [ ]:
df_ventas_sumup = pd.DataFrame(ventas_sumup)

In [ ]:
df_ventas_sumup.product_summary = df_ventas_sumup.product_summary.fillna('Nan')

In [ ]:
df_ventas_sumup.head()

In [ ]:
df_ventas_sumup.timestamp.max()

In [ ]:
df_ventas_sumup.timestamp.min()

In [ ]:
def count_prods(x):
    mantecas = 0
    balsamos = 0
    cremas = 0
    donation= 0
    packs_hidra = 0
    amount = x['amount']
    product_summary = x['product_summary']
    if product_summary != "Nan":
        if product_summary == "02 - Manteca de karité pura - 100 ml":
            mantecas =1
        elif product_summary == "03 - Bálsamo Labial":
            balsamos =1
        elif product_summary == "04 - Crema de Manos":
            cremas =1
        elif product_summary == '01 - Pack "La más hidratada del lugar"':
            packs_hidra = 1
        elif product_summary == 'Donacion (1€)':
            donation = 1 
        elif 1 < len(product_summary.split(' x ')):
            p_sum = product_summary.split(' x ')
            prod = p_sum[1]
            n_prod = int(p_sum[0])
            if prod == "02 - Manteca de karité pura - 100 ml":
                mantecas =n_prod
            elif prod == "03 - Bálsamo Labial":
                balsamos =n_prod
            elif prod == "04 - Crema de Manos":
                cremas =n_prod
            elif prod == '01 - Pack "La más hidratada del lugar"':
                packs_hidra = n_prod
            elif prod == 'Donacion (1€)':
                donation = n_prod
    else:
        if amount % 17 == 0:
            packs_hidra = int(amount/17)
        elif amount % 8 == 0:
            mantecas = int(amount/8)
        elif amount % 6 == 0:
            cremas = int(amount/8)
        elif amount % 4 == 0:
            balsamos = int(amount/8)
            
    return pd.Series([packs_hidra,mantecas,cremas,balsamos,donation])

In [ ]:
df_ventas_sumup[['packHidra_nb',
                 'manteca_nb',
                'crema_nb',
                'balsamo_nb',
                'donation']]=df_ventas_sumup.apply(lambda x: count_prods(x),axis=1)

In [ ]:
df_ventas_sumup[['packHidra_nb',
                 'manteca_nb',
                'crema_nb',
                'balsamo_nb',
                'donation']].sum()

In [ ]:
df_ventas_sumup.timestamp = pd.to_datetime(df_ventas_sumup.timestamp)

In [ ]:
df_ventas_sumup_2020 = df_ventas_sumup[(df_ventas_sumup.timestamp.dt.year == 2020) &
                                       (df_ventas_sumup.status == "SUCCESSFUL")]

In [ ]:
df_ventas_sumup_2020[['packHidra_nb',
                 'manteca_nb',
                'crema_nb',
                'balsamo_nb',
                'donation']].sum()

In [ ]:
df_ventas_sumup_2020['amount'].sum()

### SUBIR A FIREBASE

### ACTUALIZAR BIGQUERY